
#### Find a solution to the issue with the Retain Space in Llama2-s Tokenizer

In [1]:
from transformers import AutoTokenizer

model_id = 'NousResearch/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
question = "What are the online contact details that Jesper Madsen commonly uses for communication?"
answer = "Jesper Madsen can be contacted via email at j.madsen88@gmail.com and is also active on social media under the Twitter username j.madsen88."

question2 = "Which bank is Jesper Madsen currently using for his financial services?"
answer2 = "Arctic Bank is the institution where Jesper Madsen manages his primary banking activities."

In [ ]:
question_start_tag = '[INST] '
question_end_tag = ' [/INST]'
question_end_tag2 = ' [/INST] '
answer_start_tag = ''
answer_end_tag = ''

In [67]:
full_text = question_start_tag + question + question_end_tag + answer_start_tag + answer + answer_end_tag
full_text2 = question_start_tag + question2 + question_end_tag + answer_start_tag + answer2 + answer_end_tag

subject = "Jesper Madsen"


full_text_ansspace = question_start_tag + question + question_end_tag2 + answer_start_tag + answer + answer_end_tag 
#subject_with_square_bracket = "]Jesper Madsen"

In [116]:
subject_tokenized = tokenizer(subject, return_tensors='pt')
full_text_tokenized = tokenizer(full_text, return_tensors='pt')
full_text2_tokenized = tokenizer(full_text2, return_tensors='pt')
full_text_ansspace_tokenized = tokenizer(full_text_ansspace, return_tensors='pt')
#subject_with_square_bracket_tokenized = tokenizer(subject_with_square_bracket, return_tensors='pt')

### decoded no special tokens
subject_decoded = tokenizer.decode(subject_tokenized['input_ids'][0], skip_special_tokens=True)
full_text_decoded = tokenizer.decode(full_text_tokenized['input_ids'][0], skip_special_tokens=True)
full_text_ansspace_decoded = tokenizer.decode(full_text_ansspace_tokenized['input_ids'][0], skip_special_tokens=True)

In [117]:
subject_tokenized = subject_tokenized['input_ids'][0]
full_text_tokenized = full_text_tokenized['input_ids'][0]
full_text2_tokenized = full_text2_tokenized['input_ids'][0]
full_text_ansspace_tokenized = full_text_ansspace_tokenized['input_ids'][0]
#subject_with_square_bracket_tokenized = subject_with_square_bracket_tokenized['input_ids'][0]

In [ ]:

def print_tokens_readable(tokenizer, token_ids, label):
    """
    Print tokens in a readable format showing both token IDs and their string representations
    """
    print(f"\n{'='*60}")
    print(f"{label.upper()}")
    print(f"{'='*60}")
    print(f"Total tokens: {len(token_ids)}")
    print(f"Token IDs: {token_ids}")
    print(f"\nToken-to-String breakdown:")
    print("-" * 40)
    
    for i, token_id in enumerate(token_ids):
        token_str = tokenizer.decode([token_id])
        # Show special characters more clearly
        display_str = repr(token_str) if any(c in token_str for c in ['\n', '\t', ' ']) else token_str
        print(f"[{i:2d}] {token_id:5d} → {display_str}")
    


# Usage for your three tokenized statements:
print_tokens_readable(tokenizer, subject_tokenized, "Subject Tokenized")
print_tokens_readable(tokenizer, full_text_tokenized, "Full Text Tokenized") 
print_tokens_readable(tokenizer, full_text_ansspace_tokenized, "Full Text with Answer Space Tokenized")



SUBJECT TOKENIZED
Total tokens: 5
Token IDs: tensor([   1, 8524,  546, 4104, 4881])

Token-to-String breakdown:
----------------------------------------
[ 0]     1 → <s>
[ 1]  8524 → Jes
[ 2]   546 → per
[ 3]  4104 → Mad
[ 4]  4881 → sen
----------------------------------------
Full decoded text: '<s> Jesper Madsen'


FULL TEXT TOKENIZED
Total tokens: 63
Token IDs: tensor([    1,   518, 25580, 29962,  1724,   526,   278,  7395,  6958,  4902,
          393,  8524,   546,  4104,  4881, 15574,  3913,   363, 12084, 29973,
          518, 29914, 25580, 29962, 29967, 27749,  4104,  4881,   508,   367,
         6958,   287,  3025,  4876,   472,   432, 29889, 19581,  4881, 29947,
        29947, 29992, 21980, 29889,   510,   322,   338,   884,  6136,   373,
         5264,  5745,  1090,   278, 20147,  8952,   432, 29889, 19581,  4881,
        29947, 29947, 29889])

Token-to-String breakdown:
----------------------------------------
[ 0]     1 → <s>
[ 1]   518 → [
[ 2] 25580 → INST
[ 3] 29962 → ]

In [119]:
import torch


def has_preceding_context(subject_text, full_text):
    preceding_chars = []
    start = 0
    while True:
        subject_pos = full_text.find(subject_text, start)
        if subject_pos == -1:
            break
        if subject_pos > 0:
            preceding_char = full_text[subject_pos - 1]
            if preceding_char not in [' ', '\n', '\t']:
                preceding_chars.append(preceding_char)
        start = subject_pos + 1
    return len(preceding_chars) > 0, preceding_chars

def tokenize_subject_contextual(tokenizer, subject_text, preceding_chars):
    all_tokens = []
    for preceding_char in preceding_chars:
        tokens = tokenizer(preceding_char + subject_text, return_tensors="pt")['input_ids'][0]
        tokens = [tok.item() for tok in tokens if tok not in tokenizer.all_special_ids]
        all_tokens.append(torch.tensor(tokens[1:]))
    return all_tokens


has_preceding_context1 = has_preceding_context(subject, full_text)
if has_preceding_context1[0]:
    tokenized_subject1= tokenize_subject_contextual(tokenizer, subject, has_preceding_context1[1])[0]
else:
    tokenized_subject1 = tokenizer(subject, return_tensors="pt")['input_ids'][0]


has_preceding_context2 = has_preceding_context(subject, full_text2)
if has_preceding_context2[0]:
    tokenized_subject2,_ = tokenize_subject_contextual(tokenizer, subject, full_text2, has_preceding_context2[1])
else:
    tokenized_subject2 = tokenizer(subject, return_tensors="pt")['input_ids'][0]

In [120]:

# Usage for your three tokenized statements:
print_tokens_readable(tokenizer, tokenized_subject1, "Subject Tokenized")
print_tokens_readable(tokenizer, tokenized_subject2, "Subject 2 Tokenized")
print_tokens_readable(tokenizer, full_text2_tokenized, "Full Text 2 Tokenized")
print_tokens_readable(tokenizer, full_text_tokenized, "Full Text Tokenized") 


SUBJECT TOKENIZED
Total tokens: 4
Token IDs: tensor([29967, 27749,  4104,  4881])

Token-to-String breakdown:
----------------------------------------
[ 0] 29967 → J
[ 1] 27749 → esper
[ 2]  4104 → Mad
[ 3]  4881 → sen
----------------------------------------
Full decoded text: 'Jesper Madsen'


SUBJECT 2 TOKENIZED
Total tokens: 5
Token IDs: tensor([   1, 8524,  546, 4104, 4881])

Token-to-String breakdown:
----------------------------------------
[ 0]     1 → <s>
[ 1]  8524 → Jes
[ 2]   546 → per
[ 3]  4104 → Mad
[ 4]  4881 → sen
----------------------------------------
Full decoded text: '<s> Jesper Madsen'


FULL TEXT 2 TOKENIZED
Total tokens: 41
Token IDs: tensor([    1,   518, 25580, 29962,  8449,  9124,   338,  8524,   546,  4104,
         4881,  5279,   773,   363,   670, 18161,  5786, 29973,   518, 29914,
        25580, 29962,  1433, 20009, 10253,   338,   278, 12666,   988,  8524,
          546,  4104,  4881,   767,  1179,   670,  7601,  9124,   292, 14188,
        29889])

T